##### 중요 링크 : https://github.com/lovit/python_ml4tm/blob/b4b7ff9693355fb4ac342f4728606396bc81fc0d/day04_embedding_and_visualizing/day4_0_word2vec_and_doc2vec_moviereview.ipynb

In [1]:
import gensim
print('Gensim version = {}'.format(gensim.__version__))

import warnings
warnings.filterwarnings('ignore')

Gensim version = 3.8.3


##### Doc2Vec 학습
- Doc2Vec은 학습을 하기 위해서, 각각 문서의 label이 저장되어야 함
- 이를 위하여 TaggedDcoument라는 클래스가 이용됩니다.
- TaggedDocument는 단어들을 words에 , 레이블 정보를 tags에 리스트 형태로 입력합니다.

In [6]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from lovit_textmining_dataset.navermovie_comments import get_movie_comments_path


In [4]:
class Word2VecComments:
    def __init__(self, path, verbose=False):
        self.path = path
        self.verbose = verbose
        self.n_iter = 0

    def __iter__(self):
        # <idx, texts, rates>
        with open(self.path, encoding='utf-8') as f:
            for i, doc in enumerate(f):
                if self.verbose and (i % 10000 == 0):
                    print('\riter={}, sents={} ...'.format(self.n_iter, i), end='')
                yield self._tokenize(doc)
            if self.verbose:
                print('\riter={}, sents={} done'.format(self.n_iter, i))
            self.n_iter += 1

    def _tokenize(self, doc):
        idx, text, rate = doc.strip().split('\t')
        return text.split()

In [7]:
class Doc2VecComments(Word2VecComments):
    def _tokenize(self, doc):
        idx, text, rate = doc.strip().split('\t')
        

        
        return TaggedDocument(
                words = text.split(), tags = ['%s' % idx]
        )
    
path = get_movie_comments_path(large = 'large', tokenize = 'soynlp_unsup')


In [8]:
doc2vec_corpus = Doc2VecComments(path)


In [9]:
for i, doc in enumerate(doc2vec_corpus):
    if i > 3: break
    print(doc)

TaggedDocument(['명불허전'], ['72523'])
TaggedDocument(['왠지', '고사', '피의', '중간', '고사', '보다', '재미', '가', '없을듯', '해요', '만약', '보게', '된다면', '실망', '할듯'], ['72523'])
TaggedDocument(['티아라', '사랑', '해', 'ㅜ'], ['72523'])
TaggedDocument(['황정음', '윤시윤', '지붕킥', '인연', '김수로', '티아라', '지연', '공부의신', '인연', '너무', '너무', '재미', '있어요'], ['72523'])


##### 위 데이터들을 토대로 , Doc2Vec 학습

In [11]:
doc2vec_model = Doc2Vec(doc2vec_corpus)

##### 학습이 잘 되었는지 확인
- 이탈자도 상당히잘 잡아내는 것을 볼 수 있습니다 .

In [14]:
doc2vec_model.wv.most_similar('영화', topn=10)

[('애니', 0.6978278160095215),
 ('영회', 0.6964375972747803),
 ('양화', 0.6541847586631775),
 ('여화', 0.6242748498916626),
 ('애니메이션', 0.6152396202087402),
 ('엉화', 0.6064423322677612),
 ('영하', 0.5974350571632385),
 ('sf영화', 0.5960612297058105),
 ('영호ㅏ', 0.5897490978240967),
 ('영화였고', 0.5807294845581055)]

In [15]:
doc2vec_model.wv.most_similar('디카프리오', topn=10)

[('톰하디', 0.8295624256134033),
 ('브래드피트', 0.8177326917648315),
 ('앤헤서웨이', 0.8141579627990723),
 ('니콜라스홀트', 0.8040615320205688),
 ('효진이', 0.7884596586227417),
 ('윌스미스', 0.7769246697425842),
 ('앤해서웨이', 0.7761462330818176),
 ('레오', 0.764072060585022),
 ('프레디', 0.7637592554092407),
 ('로다주', 0.7597395777702332)]

##### Doc2Vec model의 dovecs 안에는 document vector와 관련된 정보들이 저장되어 있음

In [16]:
len(doc2vec_model.docvecs)

172

##### doctags
- offset: document vector의 임베딩 메트릭스의 row id
- word_count :  각 태그에 해당하는 문서에 단어가 몇 개 있었는지
- doc_count는 각 태그에 해당하는 문서가 몇번 등장하였는지 볼 수 있다.

In [21]:
doctags = doc2vec_model.docvecs.doctags.items()
doctags = sorted(doctags, key=lambda x:x[1].offset)
doctags[:3]

[('72523', Doctag(offset=0, word_count=89878, doc_count=10187)),
 ('59845', Doctag(offset=1, word_count=139795, doc_count=13095)),
 ('109753', Doctag(offset=2, word_count=200116, doc_count=10361))]

>- #72523 은 offset=0 은 docvec 에서의 row id 가 0 라는 의미입니다.



### doc2vec 해석하기

In [25]:
from lovit_textmining_dataset.navermovie_comments import load_id_to_movie

idx_to_movie = load_id_to_movie()

In [58]:
def as_name(similar):
    idx = similar[0]
    return (idx_to_movie.get(idx, 'unknown'), idx, similar[1])

print('라라랜드\n')

for similar in doc2vec_model.docvecs.most_similar('134963'):
    print(as_name(similar))

라라랜드

('비긴 어게인', '96379', 0.9099785685539246)
('어바웃 타임', '92075', 0.8202555179595947)
('인턴', '118917', 0.730941891670227)
('인사이드 아웃', '115622', 0.7061700820922852)
('뷰티 인사이드', '129050', 0.6823891401290894)
('레미제라블', '89755', 0.678883969783783)
('님아, 그 강을 건너지 마오', '130013', 0.677444338798523)
('겨울왕국', '100931', 0.6713787317276001)
('어거스트 러쉬', '66158', 0.6653317213058472)
('시간을 달리는 소녀', '63513', 0.6627459526062012)


In [59]:
print('관상\n')
for similar in doc2vec_model.docvecs.most_similar('93728'):
    print(as_name(similar))

관상

('역린', '108225', 0.8570507168769836)
('광해, 왕이 된 남자', '83893', 0.8461612462997437)
('군도:민란의 시대', '99752', 0.7632333636283875)
('사도', '121922', 0.7572081089019775)
('도둑들', '78726', 0.732923150062561)
('의형제', '52548', 0.7045021057128906)
('밀정', '137952', 0.701526403427124)
('검사외전', '130903', 0.6976866126060486)
('암살', '121048', 0.6912961006164551)
('감시자들', '98146', 0.6792099475860596)


In [60]:
print('광해 왕이된 남자\n')

for similar in doc2vec_model.docvecs.most_similar('83893'):
    print(as_name(similar))

광해 왕이된 남자

('관상', '93728', 0.8461612462997437)
('의형제', '52548', 0.7639151811599731)
('라디오 스타', '58088', 0.7365875244140625)
('역린', '108225', 0.7017911672592163)
('파파로티', '85640', 0.6967229843139648)
('사도', '121922', 0.6844784021377563)
('완득이', '80866', 0.6789095401763916)
('반창꼬', '91045', 0.6772838830947876)
('왕의 남자', '39894', 0.6753106117248535)
('시라노; 연애조작단', '73318', 0.6747324466705322)


In [62]:
print('아바타\n')
for similar in doc2vec_model.docvecs.most_similar('62266'):
    print(as_name(similar))

아바타

('트랜스포머', '61521', 0.8483377695083618)
('디스트릭트 9', '64129', 0.8433082699775696)
('2012', '49727', 0.8169445395469666)
('퍼시픽 림', '86867', 0.7704821825027466)
('스카이라인', '76581', 0.7693696022033691)
('인셉션', '52515', 0.7664300799369812)
('그래비티', '47370', 0.7574100494384766)
('트랜스포머: 패자의 역습', '68052', 0.7387325763702393)
('트랜스포머 3', '70241', 0.7289350032806396)
('다크 나이트', '62586', 0.7180948853492737)
